In [29]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [30]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight
print(params[1].size())  # conv1's .bias

input = torch.randn(1, 1, 32, 32)  # If you have a single sample, just use input.unsqueeze(0) to add a fake batch dimension.
out = net(input)
print(out)

net.zero_grad()
out.backward(torch.randn(1, 10))

10
torch.Size([6, 1, 3, 3])
torch.Size([6])
tensor([[-0.1038, -0.0730, -0.0341,  0.0880, -0.0644, -0.0364,  0.0434,  0.0780,
          0.0952, -0.0703]], grad_fn=<AddmmBackward>)


In [31]:
output = net(input)
target = torch.randn(1, 10)
criterion = nn.MSELoss()
loss = criterion(output, target)
print(loss)

print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0])
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])

tensor(1.1076, grad_fn=<MseLossBackward>)


In [32]:
net.zero_grad()     # zeroes the gradient buffers of all parameters
print(net.conv1.bias)
print(net.conv1.bias.grad)
loss.backward()
print(net.conv1.bias.grad)

Parameter containing:
tensor([ 0.1164,  0.2818, -0.3273, -0.0094,  0.2281,  0.0535],
       requires_grad=True)
tensor([0., 0., 0., 0., 0., 0.])
tensor([-0.0112, -0.0052, -0.0262, -0.0228,  0.0162, -0.0205])


In [33]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)
print(net.conv1.bias)

Parameter containing:
tensor([ 0.1165,  0.2819, -0.3270, -0.0092,  0.2280,  0.0537],
       requires_grad=True)


In [34]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers - this is the same as net.zero_grad() if all the net parameters are in the optimizer
output = net(input)
loss = criterion(output, target)
loss.backward()
print(net.conv1.bias.grad)
optimizer.step()    # Does the update
print(net.conv1.bias)

tensor([-0.0133, -0.0123, -0.0285, -0.0376,  0.0197, -0.0239])
Parameter containing:
tensor([ 0.1167,  0.2820, -0.3267, -0.0088,  0.2278,  0.0539],
       requires_grad=True)
